# Chapter 3 Autencoder



In [23]:
import numpy as np

from tensorflow.keras import datasets
from tensorflow.keras import  layers, models, datasets, callbacks
import tensorflow.keras.backend as K

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [39]:
def preprocess(imgs):
    imgs = imgs.astype("float32")/255.0 # convert to 0-1 values
    imgs = np.pad(imgs, ((0, 0), (2, 2),(2, 2)), constant_values=0.0) # pad to 32 by 32
    imgs = np.expand_dims(imgs, -1)
    return imgs

x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [34]:
# Build encoder

encoder_input = layers.Input(shape=(32,32,1), name="encoder_input")

# start creating conv encoder
x = layers.Conv2D(32, (3,3), strides=2, activation="relu", padding="same")(encoder_input)
x = layers.Conv2D(64, (3,3), strides=2, activation="relu", padding="same")(x) # 64 filters
x = layers.Conv2D(128, (3,3), strides=2, activation="relu", padding="same")(x)# 128 filters

shape_before_flattening = K.int_shape(x)[1:]

x = layers.Flatten()(x)
encoder_output = layers.Dense(2, name="encoder_output")(x)

encoder = models.Model(encoder_input, encoder_output)

In [35]:
# Build decoder


decoder_input = layers.Input(shape=(2, ), name="decoder_input")

# start creating conv encoder
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input) # Connect to dense layer from encoder
x = layers.Reshape(shape_before_flattening)(x) # Reshape the dense input to match the shape of the encoder just before its own dense layer

x = layers.Conv2DTranspose(128, (3,3), strides=2, activation="relu", padding="same")(x)  # Stack the conv2D transpose layers on top of each other 
x = layers.Conv2DTranspose(64, (3,3), strides=2, activation="relu", padding="same")(x) 
x = layers.Conv2DTranspose(32, (3,3), strides=2, activation="relu", padding="same")(x) 

decoder_output = layers.Conv2D(1, (3, 3), strides=1, activation="sigmoid", padding="same", name="decoder_output")(x)
decoder = models.Model(decoder_input, decoder_output)

In [36]:
autoencoder = models.Model(encoder_input, decoder(encoder_output))

In [41]:
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")

In [ ]:
autoencoder.fit(x_train,
                x_train, 
                epochs=10, 
                batch_size=100, 
                shuffle=True, 
                validation_data = (x_test, x_test)
               )

Epoch 1/10
600/600 [==============================] - 87s 143ms/step - loss: 0.2912 - val_loss: 0.2610
Epoch 2/10
600/600 [==============================] - 88s 147ms/step - loss: 0.2562 - val_loss: 0.2561
Epoch 3/10
600/600 [==============================] - 89s 149ms/step - loss: 0.2530 - val_loss: 0.2536
Epoch 4/10
600/600 [==============================] - 92s 153ms/step - loss: 0.2511 - val_loss: 0.2521
Epoch 5/10
600/600 [==============================] - 92s 154ms/step - loss: 0.2500 - val_loss: 0.2514
Epoch 6/10
121/600 [=====>........................] - ETA: 1:09 - loss: 0.2491